In [2]:
import json

In [3]:
class Node:
    def __init__(self, name, is_directory=True):
        self.name = name
        self.is_directory = is_directory
        self.children = {}  # Subpastas e arquivos (chave: nome, valor: objeto Node)
        self.parent = None  # Referência ao diretório pai (importante para navegação)

    def add_child(self, child_node):
        child_node.parent = self
        self.children[child_node.name] = child_node

    def remove_child(self, child_name):
        if child_name in self.children:
            del self.children[child_name]

    def __str__(self):
        return f"{self.name} {'(dir)' if self.is_directory else '(file)'}"

class FileSystem:
    def __init__(self):
        self.root = Node("/", True)  # O diretório raiz é a pasta "/"
        self.current_node = self.root

    def ls(self):
        # Exibe o conteúdo do diretório atual
        for child in self.current_node.children.values():
            print(child)

    def resolve_path(self, path):
        if path.startswith("/"):  # Caminho absoluto
            node = self.root
            path_parts = path.split("/")[1:]  # Ignora a raiz "/"
        else:  # Caminho relativo
            node = self.current_node
            path_parts = path.split("/")

        for part in path_parts:
            if part == "..":
                if node.parent:
                    node = node.parent
            elif part in node.children and node.children[part].is_directory:
                node = node.children[part]
            else:
                print(f"Erro: {part} não encontrado ou não é um diretório.")
                return None

        return node

    def cd(self, path):
        new_node = self.resolve_path(path)
        if new_node:
            self.current_node = new_node


    def mkdir(self, dir_name):
        # Cria uma nova pasta (diretório)
        if dir_name not in self.current_node.children:
            new_dir = Node(dir_name, True)
            self.current_node.add_child(new_dir)
        else:
            print(f"O diretório {dir_name} já existe.")

    def touch(self, file_name):
        # Cria um novo arquivo
        if file_name not in self.current_node.children:
            new_file = Node(file_name, False)
            self.current_node.add_child(new_file)
        else:
            print(f"O arquivo {file_name} já existe.")
            
    def mv(self, old_name, new_name):
        if old_name in self.current_node.children:
            node = self.current_node.children[old_name]
            # Verificar se o nome novo já existe
            if new_name not in self.current_node.children:
                node.name = new_name
                self.current_node.children[new_name] = node
                del self.current_node.children[old_name]
            else:
                print(f"O nome {new_name} já está em uso.")
        else:
            print(f"{old_name} não encontrado.")
            
    def rm(self, name):
        if name in self.current_node.children:
            node = self.current_node.children[name]
            if node.is_directory and node.children:
                print(f"Erro: O diretório {name} não está vazio.")
            else:
                self.current_node.remove_child(name)
                print(f"{name} removido com sucesso.")
        else:
            print(f"{name} não encontrado.")
            
    
    def save(self, filename):
        def node_to_dict(node):
            return {
                "name": node.name,
                "is_directory": node.is_directory,
                "children": {k: node_to_dict(v) for k, v in node.children.items()}
            }

        with open(filename, 'w') as f:
            json.dump(node_to_dict(self.root), f)

    def save_to_file(self, filename="filesystem.json"):
        self.save(filename)
        print(f"Sistema de arquivos salvo em {filename}.")
        
    def load(self, filename):
        def dict_to_node(data):
            node = Node(data['name'], data['is_directory'])
            node.children = {k: dict_to_node(v) for k, v in data['children'].items()}
            return node

        with open(filename, 'r') as f:
            data = json.load(f)
            self.root = dict_to_node(data)
            self.current_node = self.root

    def load_from_file(self, filename="filesystem.json"):
        self.load(filename)
        print(f"Sistema de arquivos carregado de {filename}.")


    def tree(self, node=None, indent=""):
        if node is None:
            node = self.current_node

        print(indent + node.name)
        indent += "    "
        for child in node.children.values():
            if child.is_directory:
                self.tree(child, indent)
            else:
                print(indent + child.name)





In [4]:
fs = FileSystem()

# Criação de diretórios e arquivos
fs.mkdir("home")
fs.cd("home")
fs.mkdir("user")
fs.touch("file1.txt")
fs.cd("user")
fs.touch("file2.txt")

# Listar arquivos e navegação
fs.ls()  # Lista o conteúdo da pasta user
fs.cd("..")
fs.tree()  # Exibe a estrutura completa da árvore

# Renomear e deletar
fs.mv("file1.txt", "file_renamed.txt")
fs.rm("user")

# Salvar e carregar estado
fs.save_to_file("filesystem.json")
fs.load_from_file("filesystem.json")

file2.txt (file)
home
    user
        file2.txt
    file1.txt
Erro: O diretório user não está vazio.
Sistema de arquivos salvo em filesystem.json.
Sistema de arquivos carregado de filesystem.json.
